## LTSM

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../embeddings/BERT_Embeddings_full.csv')
df.head()

,age,age_encoded,word_tokens,lemma,0,1,2,3,4,5,...,758,759,760,761,762,763,764,765,766,767
0,a2,1,"['quantos', 'empregados', 'em', 'cada', 'um', ...",quanto empregado em cada um de o atual nível s...,0.564466,0.815525,0.107565,0.319572,0.101842,0.105100,...,0.275671,0.219380,0.934174,0.909270,0.090046,0.897559,0.590714,0.291341,0.416531,0.900556
1,a2,1,"['solicito', 'cópia', 'das', 'atas', 'do', 'co...",solicito cópia de o ata de o conselho de admin...,0.520970,0.815053,0.120955,0.300525,0.079995,0.149684,...,0.282352,0.235483,0.916027,0.947861,0.102334,0.876205,0.603263,0.254797,0.394609,0.841805
2,a2,1,"['solicito', 'informar', 'a', 'norma', 'lei', ...",solicito informar o norma lei decreto portaria...,0.535018,0.845178,0.106620,0.330766,0.061033,0.094648,...,0.288288,0.232071,0.927528,0.953746,0.102037,0.927873,0.533769,0.266102,0.429789,0.895740
3,a2,1,"['solicito', 'por', 'gentileza', 'a', 'informa...",solicito por gentileza o informação sobre o qu...,0.548547,0.784378,0.057764,0.275971,0.123137,0.096042,...,0.306542,0.244469,0.954206,0.941051,0.075034,0.899374,0.585249,0.279545,0.380809,0.939743
4,a2,1,"['solicito', 'por', 'gentileza', 'a', 'informa...",solicito por gentileza o informação sobre o qu...,0.559063,0.799921,0.060454,0.274691,0.135161,0.079634,...,0.304113,0.247853,0.955085,0.938762,0.081348,0.894296,0.595016,0.278718,0.383074,0.934677


In [3]:
df.groupby('age')['age_encoded'].count()

age
a1    1200
a2    2000
a3    3000
a4    2000
Name: age_encoded, dtype: int64

In [4]:
df_label = pd.get_dummies(df['age'])
df_text = df.drop(['word_tokens', 'lemma', 'age', 'age_encoded'], axis = 1)

In [5]:
#df.groupby('age_encoded').describe()

## Model

In [9]:
import tensorflow as tf
#from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [10]:
max_features = 700
embed_dim = 32

In [11]:
batch_size = 32
early_stopping = EarlyStopping(monitor='val_loss', patience= 3)

In [12]:
def modelLSTMPos(df_shape):
    
    model = Sequential([
        layers.Embedding(max_features, embed_dim, input_length= df_shape.shape[1]),
        layers.Conv1D(filters=32, kernel_size=3, padding='same', activation= 'relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.LSTM(64, dropout=0.4, recurrent_dropout=0.2),
        layers.Dense(64, activation='relu'),
        layers.Dense(4, activation='softmax')
    ])

    #optimizer = Adam(learning_rate=3e-4)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [13]:
model = modelLSTMPos(df)

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 772, 32)           22400     
                                                                 
 conv1d (Conv1D)             (None, 772, 32)           3104      
                                                                 
 max_pooling1d (MaxPooling1  (None, 386, 32)           0         
 D)                                                              
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 4)                 260       
                                                        

### BERT - Train, Test split

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier

2023-12-14 01:40:39.563540: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-14 01:40:39.749322: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-14 01:40:39.749347: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-14 01:40:39.750428: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-14 01:40:39.865479: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-14 01:40:39.866392: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df_text, df_label, train_size=0.8, random_state=42 )
X_train.shape


In [15]:
model = modelLSTMPos(df_text)

In [16]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
205/205 [==============================] - 33s 153ms/step - loss: 0.5567 - accuracy: 0.3681 - val_loss: 0.5477 - val_accuracy: 0.3549
Epoch 2/10
205/205 [==============================] - 35s 169ms/step - loss: 0.5468 - accuracy: 0.3660 - val_loss: 0.5488 - val_accuracy: 0.3549
Epoch 3/10
205/205 [==============================] - 34s 167ms/step - loss: 0.5465 - accuracy: 0.3686 - val_loss: 0.5478 - val_accuracy: 0.3549
Epoch 4/10
205/205 [==============================] - 42s 203ms/step - loss: 0.5465 - accuracy: 0.3686 - val_loss: 0.5471 - val_accuracy: 0.3549
Epoch 5/10
205/205 [==============================] - 40s 194ms/step - loss: 0.5465 - accuracy: 0.3686 - val_loss: 0.5488 - val_accuracy: 0.3549
Epoch 6/10
205/205 [==============================] - 39s 189ms/step - loss: 0.5468 - accuracy: 0.3686 - val_loss: 0.5480 - val_accuracy: 0.3549
Epoch 7/10
205/205 [==============================] - 39s 189ms/step - loss: 0.5464 - accuracy: 0.3686 - val_loss: 0.5473 - val_ac

### BERT - K-fold

In [17]:
#from sklearn.model_selection import KFold

In [18]:
# clf = KerasClassifier(build_fn=modelLSTMPos(df_text), epochs=10, batch_size=32, verbose=1)
# resultPos = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=10)
# print(resultPos)
# resultPos.mean()

### Plain text - Train, test

In [19]:

df_plain = pd.read_csv('../data/enc_tok_nostopword_nopunct_lemm.csv')
df_plain.head()

,Unnamed: 0,req_text,age,age_encoded,word_tokens,lemma
0,0,quantos empregados em cada um dos atuais níve...,a2,1,"['quantos', 'empregados', 'cada', 'atuais', 'n...",quanto empregado cada atual nível salarial pca...
1,1,solicito cópia das atas do conselho de admini...,a2,1,"['solicito', 'cópia', 'atas', 'conselho', 'adm...",solicito cópia ata conselho administração bnde...
2,2,solicito informar a norma lei decreto portari...,a2,1,"['solicito', 'informar', 'norma', 'lei', 'decr...",solicito informar norma lei decreto portaria i...
3,3,solicito por gentileza a informação sobre a q...,a2,1,"['solicito', 'gentileza', 'informação', 'sobre...",solicito gentileza informação sobre quantidade...
4,4,solicito por gentileza a informação sobre a q...,a2,1,"['solicito', 'gentileza', 'informação', 'sobre...",solicito gentileza informação sobre quantidade...


In [20]:
tokenizer = Tokenizer(num_words= max_features, split= ' ')
tokenizer.fit_on_texts(df_plain['lemma'].values)
x = tokenizer.texts_to_sequences(df_plain['lemma'].values)
x = pad_sequences(x)

In [21]:
y = pd.get_dummies(df_plain['age'])

In [22]:
X_treino, X_teste, y_treino, y_teste = train_test_split(x, y, train_size=0.8, random_state=42 )


In [23]:
model_plain = modelLSTMPos(x)

In [24]:
history = model_plain.fit(X_treino, y_treino, epochs=20, validation_data=(X_teste, y_teste))

Epoch 1/20
205/205 [==============================] - 10s 40ms/step - loss: 0.5557 - accuracy: 0.3578 - val_loss: 0.5401 - val_accuracy: 0.3549
Epoch 2/20
205/205 [==============================] - 8s 38ms/step - loss: 0.5125 - accuracy: 0.4183 - val_loss: 0.5007 - val_accuracy: 0.4378
Epoch 3/20
205/205 [==============================] - 8s 38ms/step - loss: 0.4824 - accuracy: 0.4747 - val_loss: 0.5011 - val_accuracy: 0.4506
Epoch 4/20
205/205 [==============================] - 8s 37ms/step - loss: 0.4646 - accuracy: 0.5046 - val_loss: 0.4954 - val_accuracy: 0.4561
Epoch 5/20
205/205 [==============================] - 8s 38ms/step - loss: 0.4480 - accuracy: 0.5355 - val_loss: 0.4957 - val_accuracy: 0.4604
Epoch 6/20
205/205 [==============================] - 8s 38ms/step - loss: 0.4339 - accuracy: 0.5582 - val_loss: 0.5052 - val_accuracy: 0.4585
Epoch 7/20
205/205 [==============================] - 8s 38ms/step - loss: 0.4165 - accuracy: 0.5849 - val_loss: 0.5207 - val_accuracy: 0.454

##